## Notebook 6
# Recomendación

## Se pide:

Se deberán probar todos los modelos posibles para cada uno de los siguientes problemas de negocio, industrializando en el paquete solamente el mejor de ellos para cada uno de los casos, con los mejores hiperparámetros y transformaciones previas encontradas.

<b>Recomendación</b>:<br>
Crear un recomendador que dado un vídeo te recomiende vídeos parecidos.

## Importaciones

In [1]:
import pandas as pd
import numpy as np

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp

import string
import nltk
from nltk.corpus import stopwords

## Carga de datos
Carga del fichero de datos generado al final del EDA.

In [2]:
df = pd.read_csv('..//data//processed//csv_eda.csv', index_col=0)

# Crear un recomendador de vídeos parecidos

A partir del dataset tras el EDA vamos a buscar los vídeos más similares a uno dado.

# KNN (K Nearest Neighbors)

### Preprocesamiento de los datos

<b>Selección de variables</b>

Para estudiar la similitud entre vídeos, vamos a utilizar las variables:
- title
- views
- likes
- dislikes
- comment_count
- category
- ratio_viralidad
- ratio_interaccion_dia

In [3]:
df.columns

Index(['title', 'channel_title', 'tags', 'views', 'likes', 'dislikes',
       'comment_count', 'comments_disabled', 'ratings_disabled', 'category',
       'country', 'trending_date_year', 'trending_date_month',
       'publish_time_year', 'publish_time_month', 'ratio_viralidad',
       'ratio_interaccion_dia', 'publish_trending_days',
       'channel_title_encoded', 'tags_encoded', 'category_encoded',
       'country_encoded', 'views_encoded', 'likes_encoded', 'dislikes_encoded',
       'comment_count_encoded', 'ratio_interaccion_dia_encoded',
       'trending_date_year_encoded', 'trending_date_month_encoded',
       'publish_time_year_encoded', 'publish_time_month_encoded'],
      dtype='object')

In [4]:
df_recom = df[['title', 'views_encoded', 'likes_encoded', 'dislikes_encoded', 'comment_count_encoded',\
              'category_encoded', 'ratio_viralidad', 'ratio_interaccion_dia_encoded']].copy()

df_recom.head()

,title,views_encoded,likes_encoded,dislikes_encoded,comment_count_encoded,category_encoded,ratio_viralidad,ratio_interaccion_dia_encoded
0,Eminem - Walk On Water (Audio) ft. Beyoncé,2.195188,4.501606,1.803614,4.773962,8,0.055758,4.334115
1,PLUSH - Bad Unboxing Fan Mail,-0.046118,0.536588,-0.020748,0.342217,1,0.140454,0.455009
2,"Racist Superman | Rudy Mancuso, King Bach & Le...",0.256091,0.646234,0.138859,0.151795,1,0.049995,0.536320
3,I Dare You: GOING BALD!?,0.103985,0.563307,-0.007590,0.518462,3,0.072404,0.499064
4,Ed Sheeran - Perfect (Official Music Video),4.467192,9.591119,0.827083,3.171140,8,0.051912,8.064620


<b>title</b>

Es la única variable categórica. Vamos a convertirla en una matriz numérica usando TF-IDF, lo que nos será útil para el algoritmo de recomendaciones.

In [5]:
# quitamos palabras comunes en varios idiomas
stop_words_en = stopwords.words('english')
stop_words_fr = stopwords.words('french')
stop_words_ge = stopwords.words('german')
stop_words_be = stopwords.words('bengali')
stop_words_sp = stopwords.words('spanish')
stop_words_ru = stopwords.words('russian')

stop_words_multi = list(set(stop_words_en + stop_words_fr + stop_words_ge + stop_words_be + stop_words_sp + stop_words_ru))

In [6]:
# quitamos también palabras no imprimibles (tenemos datos coreanos o bengalíes, por ejemplo)
def is_latin(word):
    return all(char in string.printable for char in word)

filtered_stop_words_multi = [word for word in stop_words_multi if is_latin(word)]

In [7]:
# convertimos 'title' en una matriz TF-IDF de dimensión (n_samples, n_features)
vectorizer_title = TfidfVectorizer(stop_words=filtered_stop_words_multi)
X_titles = vectorizer_title.fit_transform(df_recom['title'])

print(X_titles.shape)

(362283, 139948)


In [8]:
# unimos todas las características en una única matriz dispersa, compuesta de vectores a partir de esas características
X = sp.hstack((X_titles, \
               sp.csr_matrix(df_recom['views_encoded'].values.reshape(-1, 1)), \
               sp.csr_matrix(df_recom['likes_encoded'].values.reshape(-1, 1)),\
               sp.csr_matrix(df_recom['dislikes_encoded'].values.reshape(-1, 1)),\
               sp.csr_matrix(df_recom['comment_count_encoded'].values.reshape(-1, 1)),\
               sp.csr_matrix(df_recom['category_encoded'].values.reshape(-1, 1)),\
               sp.csr_matrix(df_recom['ratio_viralidad'].values.reshape(-1, 1)),\
               sp.csr_matrix(df_recom['ratio_interaccion_dia_encoded'].values.reshape(-1, 1)) ))

### NearestNeighbors

In [9]:
# entrenamos el modelo
nn = NearestNeighbors(n_neighbors=5, metric='cosine')

nn.fit(X)

NearestNeighbors(metric='cosine')

## Recomendador

Función para buscar las recomendaciones (los vecinos más cercanos a un registro dado).

In [10]:
def recomendar_videos(titulo, modelo, df, vectorizer, n_neighbors=5):
    '''
    Busca los vecinos más cercanos.
    :titulo: título por el que haremos la búsqueda
    :model: instancia del modelo a utilizar
    :df: dataframe con los datos donde buscar
    :vectorizer: instancia del conversor del título en una matriz TF-IDF
    :n_neighbors: núm. vecinos a buscar (5 por defecto)
    :return: tupla (índice, distancia) con los índices del dataframe y las distancias de los vecinos más cercanos
    '''
    # aplicamos el conversor al título a buscar
    titulo_vectorizado = vectorizer.transform([titulo])
    
    # añadimos los valores promedio al resto de características
    dummy_views = sp.csr_matrix([[df['views_encoded'].mean()]])
    dummy_likes = sp.csr_matrix([[df['likes_encoded'].mean()]])
    dummy_dislikes = sp.csr_matrix([[df['dislikes_encoded'].mean()]])
    dummy_comment_count = sp.csr_matrix([[df['comment_count_encoded'].mean()]])
    dummy_category = sp.csr_matrix([[df['category_encoded'].mean()]])
    dummy_ratio_viralidad = sp.csr_matrix([[df['ratio_viralidad'].mean()]])
    dummy_ratio_interaccion_dia = sp.csr_matrix([[df['ratio_interaccion_dia_encoded'].mean()]])
    
    # creamos el vector completo a buscar
    vector_completo = sp.hstack((titulo_vectorizado, dummy_views, dummy_likes, dummy_dislikes, dummy_comment_count,\
                                dummy_category, dummy_ratio_viralidad, dummy_ratio_interaccion_dia))
    
    # buscamos los vecinos más cercanos
    distancias, indices = modelo.kneighbors(vector_completo, n_neighbors=n_neighbors)

    # lista de tuplas (índice, distancia) de los vecinos más cercanos
    neighbors = []
    for x1, x2 in zip(indices[0], distancias[0]):
        neighbors.append((x1, x2))

    # ordenamos por distancia de mayor a menor
    neighbors.sort(key = lambda x: x[1], reverse=True)

    return neighbors

In [11]:
titulo_ejemplo = "Baby Driver"
recomendaciones = recomendar_videos(titulo_ejemplo, nn, df_recom, vectorizer_title)

for i in recomendaciones:
    print('Vídeo', i[0], '(dist = {:.2}): '.format(i[1]), df_recom.iloc[i[0]]['title'])

Vídeo 23325 (dist = 0.011):  Baby's First Word!
Vídeo 23525 (dist = 0.011):  Baby's First Word!
Vídeo 3324 (dist = 0.0098):  OUR BABY IS FINALLY HERE!
Vídeo 33571 (dist = 0.009):  WE'RE BACK BABY!!!
Vídeo 33827 (dist = 0.0088):  WE'RE BACK BABY!!!


In [12]:
titulo_ejemplo = "Rock Music"
recomendaciones = recomendar_videos(titulo_ejemplo, nn, df_recom, vectorizer_title)

for i in recomendaciones:
    print('Vídeo', i[0], '(dist = {:.2}): '.format(i[1]), df_recom.iloc[i[0]]['title'])

Vídeo 343493 (dist = 0.0079):  Kid Rock - American Rock 'n Roll (Official Video)
Vídeo 343704 (dist = 0.0079):  Kid Rock - American Rock 'n Roll (Official Video)
Vídeo 343918 (dist = 0.0079):  Kid Rock - American Rock 'n Roll (Official Video)
Vídeo 344126 (dist = 0.0079):  Kid Rock - American Rock 'n Roll (Official Video)
Vídeo 344334 (dist = 0.0078):  Kid Rock - American Rock 'n Roll (Official Video)


In [13]:
titulo_ejemplo = "Rugby"
recomendaciones = recomendar_videos(titulo_ejemplo, nn, df_recom, vectorizer_title, n_neighbors=7)

for i in recomendaciones:
    print('Vídeo', i[0], '(dist = {:.2}): '.format(i[1]), df_recom.iloc[i[0]]['title'])

Vídeo 128877 (dist = 0.0099):  Scarlets VS Benetton. Highlights GAME Champions Cup Rugby Union. 09/12/2017
Vídeo 87200 (dist = 0.0097):  RC Toulon v Bath Rugby (P5) - Highlights – 09.12.2017
Vídeo 112234 (dist = 0.0097):  Racing 92 v Munster Rugby (SF) - Highlights - 22.04.2018
Vídeo 98076 (dist = 0.0096):  MOSCATO: RUGBY, DEBRIEF DU MATCH FRANCE-IRLANDE  05/02/18
Vídeo 96964 (dist = 0.0092):  XV de France : Priso piège du rugby !
Vídeo 86722 (dist = 0.0082):  Boxing Days Rugby | Le clip officiel
Vídeo 90829 (dist = 0.0068):  RUGBY - 2017 TOP TRIES
